In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import re

In [2]:
df=pd.read_excel("/content/Claims data_1.xlsx")
df

,LossDate,Jurisdiction,CoverageType,Deductible,Limit,InjuryCause,YearBuilt,Stories,ClassCode,ConstructionTypeDesc,ZipCode,Amount
0,2016-01-24 00:00:00.000,Pennsylvania,BOP-Building,FixedDollar - 2500.00,Standard - 814549.00,FIRE,1950.0,2.0,ClassCode:65146,JoistedMasonry,17104,105202.25
1,2016-01-24 00:00:00.000,Pennsylvania,BOP-Building,FixedDollar - 2500.00,Blanket Group 1 Blanket - 6304694.00,FIRE,1950.0,2.0,ClassCode:65146,JoistedMasonry,17104,105202.25
2,2016-02-01 00:00:00.000,Pennsylvania,BOP-Building,FixedDollar - 500.00,Standard - 400192.00,WATER,1981.0,1.0,ClassCode:64061,JoistedMasonry,17013,5026.72
3,2016-02-07 00:00:00.000,Pennsylvania,BOP-Building,FixedDollar - 1000.00,Standard - 13975864.00,FIRE,1981.0,8.0,ClassCode:65146,FireResistive,15857,5666.01
4,2016-01-23 00:00:00.000,Delaware,BOP-Building,FixedDollar - 2500.00,Standard - 810000.00,WIND,1997.0,2.0,ClassCode:65146,Frame,19709,14812.15
...,...,...,...,...,...,...,...,...,...,...,...,...
10813,2023-07-01 00:00:00.000,Dist. of Columbia,BOP-Building,FixedDollar - 2500.00,Standard - 1119319.00,WIND,1900.0,2.0,ClassCode:69145,JoistedMasonry,20003,NaN
10814,2023-07-01 00:00:00.000,Dist. of Columbia,BOP-Building,PerUnit - 2500.00,Standard - 1119319.00,WIND,1900.0,2.0,ClassCode:69145,JoistedMasonry,20003,NaN
10815,2023-08-13 00:00:00.000,Dist. of Columbia,BOP-Building,FixedDollar - 5000.00,Standard - 1397478.00,WATER,1944.0,2.0,ClassCode:65146,JoistedMasonry,20011,NaN
10816,2023-08-01 00:00:00.000,Pennsylvania,BOP-Building,FixedDollar - 2500.00,Standard - 700000.00,NaN,2021.0,1.0,ClassCode:65144,Frame,16066,NaN


In [3]:
df = df[pd.to_numeric(df['Amount'], errors='coerce').notna()]
df

,LossDate,Jurisdiction,CoverageType,Deductible,Limit,InjuryCause,YearBuilt,Stories,ClassCode,ConstructionTypeDesc,ZipCode,Amount
0,2016-01-24 00:00:00.000,Pennsylvania,BOP-Building,FixedDollar - 2500.00,Standard - 814549.00,FIRE,1950.0,2.0,ClassCode:65146,JoistedMasonry,17104,105202.25
1,2016-01-24 00:00:00.000,Pennsylvania,BOP-Building,FixedDollar - 2500.00,Blanket Group 1 Blanket - 6304694.00,FIRE,1950.0,2.0,ClassCode:65146,JoistedMasonry,17104,105202.25
2,2016-02-01 00:00:00.000,Pennsylvania,BOP-Building,FixedDollar - 500.00,Standard - 400192.00,WATER,1981.0,1.0,ClassCode:64061,JoistedMasonry,17013,5026.72
3,2016-02-07 00:00:00.000,Pennsylvania,BOP-Building,FixedDollar - 1000.00,Standard - 13975864.00,FIRE,1981.0,8.0,ClassCode:65146,FireResistive,15857,5666.01
4,2016-01-23 00:00:00.000,Delaware,BOP-Building,FixedDollar - 2500.00,Standard - 810000.00,WIND,1997.0,2.0,ClassCode:65146,Frame,19709,14812.15
...,...,...,...,...,...,...,...,...,...,...,...,...
10730,2023-07-22 00:00:00.000,Pennsylvania,BOP-Building,FixedDollar - 5000.00,Standard - 2403492.00,LIGHTNING,1916.0,2.0,ClassCode:65141,Frame,17701,100.70
10732,2023-07-29 00:00:00.000,Pennsylvania,BOP-Building,FixedDollar - 1000.00,Standard - 208403.00,WIND,1890.0,2.0,ClassCode:65141,Frame,15206,2351.46
10756,2023-08-05 00:00:00.000,Pennsylvania,BOP-Building,FixedDollar - 1000.00,Standard - 218339.00,VEHICLES,1940.0,1.0,ClassCode:09411,JoistedMasonry,17602,11500.00
10765,2023-07-29 00:00:00.000,Pennsylvania,BOP-Building,FixedDollar - 1000.00,Standard - 1394668.00,VEHICLES,1989.0,1.0,ClassCode:59999,JoistedMasonry,15146,10000.00


In [4]:
def extract_amount(value):
    match = re.search(r'(\d+\.\d+)', str(value))
    if match:
        return float(match.group(1))
    else:
        return None

In [5]:
df['Amount'] = df['Amount'].apply(extract_amount)
df = df.dropna(subset=['Amount'])
df

<ipython-input-5-45d030beab8d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Amount'] = df['Amount'].apply(extract_amount)


,LossDate,Jurisdiction,CoverageType,Deductible,Limit,InjuryCause,YearBuilt,Stories,ClassCode,ConstructionTypeDesc,ZipCode,Amount
0,2016-01-24 00:00:00.000,Pennsylvania,BOP-Building,FixedDollar - 2500.00,Standard - 814549.00,FIRE,1950.0,2.0,ClassCode:65146,JoistedMasonry,17104,105202.25
1,2016-01-24 00:00:00.000,Pennsylvania,BOP-Building,FixedDollar - 2500.00,Blanket Group 1 Blanket - 6304694.00,FIRE,1950.0,2.0,ClassCode:65146,JoistedMasonry,17104,105202.25
2,2016-02-01 00:00:00.000,Pennsylvania,BOP-Building,FixedDollar - 500.00,Standard - 400192.00,WATER,1981.0,1.0,ClassCode:64061,JoistedMasonry,17013,5026.72
3,2016-02-07 00:00:00.000,Pennsylvania,BOP-Building,FixedDollar - 1000.00,Standard - 13975864.00,FIRE,1981.0,8.0,ClassCode:65146,FireResistive,15857,5666.01
4,2016-01-23 00:00:00.000,Delaware,BOP-Building,FixedDollar - 2500.00,Standard - 810000.00,WIND,1997.0,2.0,ClassCode:65146,Frame,19709,14812.15
...,...,...,...,...,...,...,...,...,...,...,...,...
10730,2023-07-22 00:00:00.000,Pennsylvania,BOP-Building,FixedDollar - 5000.00,Standard - 2403492.00,LIGHTNING,1916.0,2.0,ClassCode:65141,Frame,17701,100.70
10732,2023-07-29 00:00:00.000,Pennsylvania,BOP-Building,FixedDollar - 1000.00,Standard - 208403.00,WIND,1890.0,2.0,ClassCode:65141,Frame,15206,2351.46
10756,2023-08-05 00:00:00.000,Pennsylvania,BOP-Building,FixedDollar - 1000.00,Standard - 218339.00,VEHICLES,1940.0,1.0,ClassCode:09411,JoistedMasonry,17602,11500.00
10765,2023-07-29 00:00:00.000,Pennsylvania,BOP-Building,FixedDollar - 1000.00,Standard - 1394668.00,VEHICLES,1989.0,1.0,ClassCode:59999,JoistedMasonry,15146,10000.00


In [6]:
def clean_zip(zip_code):
    #match = re.search(r'(\d+)', str(zip_code))
    match = re.search(r'(\d+\.\d+)', str(zip_code))
    if match:
        return match.group(1)
    else:
        return None

In [7]:
df['ZipCode'] = df['ZipCode'].apply(clean_zip)
df

,LossDate,Jurisdiction,CoverageType,Deductible,Limit,InjuryCause,YearBuilt,Stories,ClassCode,ConstructionTypeDesc,ZipCode,Amount
0,2016-01-24 00:00:00.000,Pennsylvania,BOP-Building,FixedDollar - 2500.00,Standard - 814549.00,FIRE,1950.0,2.0,ClassCode:65146,JoistedMasonry,None,105202.25
1,2016-01-24 00:00:00.000,Pennsylvania,BOP-Building,FixedDollar - 2500.00,Blanket Group 1 Blanket - 6304694.00,FIRE,1950.0,2.0,ClassCode:65146,JoistedMasonry,None,105202.25
2,2016-02-01 00:00:00.000,Pennsylvania,BOP-Building,FixedDollar - 500.00,Standard - 400192.00,WATER,1981.0,1.0,ClassCode:64061,JoistedMasonry,None,5026.72
3,2016-02-07 00:00:00.000,Pennsylvania,BOP-Building,FixedDollar - 1000.00,Standard - 13975864.00,FIRE,1981.0,8.0,ClassCode:65146,FireResistive,None,5666.01
4,2016-01-23 00:00:00.000,Delaware,BOP-Building,FixedDollar - 2500.00,Standard - 810000.00,WIND,1997.0,2.0,ClassCode:65146,Frame,None,14812.15
...,...,...,...,...,...,...,...,...,...,...,...,...
10730,2023-07-22 00:00:00.000,Pennsylvania,BOP-Building,FixedDollar - 5000.00,Standard - 2403492.00,LIGHTNING,1916.0,2.0,ClassCode:65141,Frame,None,100.70
10732,2023-07-29 00:00:00.000,Pennsylvania,BOP-Building,FixedDollar - 1000.00,Standard - 208403.00,WIND,1890.0,2.0,ClassCode:65141,Frame,None,2351.46
10756,2023-08-05 00:00:00.000,Pennsylvania,BOP-Building,FixedDollar - 1000.00,Standard - 218339.00,VEHICLES,1940.0,1.0,ClassCode:09411,JoistedMasonry,None,11500.00
10765,2023-07-29 00:00:00.000,Pennsylvania,BOP-Building,FixedDollar - 1000.00,Standard - 1394668.00,VEHICLES,1989.0,1.0,ClassCode:59999,JoistedMasonry,None,10000.00


In [8]:
df['Amount'].fillna(0, inplace=True)

# Example: Convert 'Deductible' and 'Limit' to numeric values
df['Deductible'] = df['Deductible'].str.extract(r'(\d+\.\d+)').astype(float)
df['Limit'] = df['Limit'].str.extract(r'(\d+\.\d+)').astype(float)
#df['Deductible'] = df['Deductible'].replace('FixedDollar - ', '', regex=True).astype(float)
#df['Limit'] = df['Limit'].replace('Standard - ', '', regex=True).replace('Blanket Group 1 Blanket - ', '', regex=True).astype(float)
# df['Deductible'] = df['Deductible'].str.replace('FixedDollar - ', '').astype(float)
# df['Limit'] = df['Limit'].str.replace('Standard - ', '').str.replace('Blanket Group 1 Blanket - ', '').astype(float)

# You can continue with your data preprocessing steps as needed for your predictive model.

# Split the data into features (X) and target (y)
X = df.drop(columns=["Amount"])
y = df["Amount"]

In [9]:
X = df.drop(columns=["Amount"])
y = df["Amount"]

In [10]:
reference_date = pd.to_datetime('2016-01-01')
X['LossDate'] = (pd.to_datetime(X['LossDate']) - reference_date).dt.days
X = pd.get_dummies(X, columns=["Jurisdiction", "CoverageType", "InjuryCause", "ClassCode", "ConstructionTypeDesc"])

In [118]:
from sklearn.preprocessing import StandardScaler

In [126]:
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [127]:
X_train = scaler.transform(X_train)

In [132]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)

In [144]:
X_train

array([[2.34400000e+03, 2.50000000e+03, 1.18976000e+06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.30400000e+03, 1.00000000e+03, 5.61363000e+06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.24100000e+03, 2.50000000e+03, 2.65200000e+05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [6.25000000e+02, 1.00000000e+03, 2.76383384e+08, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.65000000e+03, 1.00000000e+03, 5.75707500e+06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.99500000e+03, 2.50000000e+03, 1.21247360e+07, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

**Linear Regression**

In [133]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

In [134]:
imputer_X = SimpleImputer(strategy='mean')
X = imputer_X.fit_transform(X)
imputer_y = SimpleImputer(strategy='mean')
if y.ndim == 1:
    y = y.values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [135]:
y_pred

array([[-1794.42340293],
       [43892.33606549],
       [18557.15329691],
       ...,
       [43158.72056827],
       [-1520.17085149],
       [35540.00968771]])

In [136]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
rmse

168868.39456997858

In [131]:
R2= r2_score(y_test, y_pred)
R2

0.0748720488451372

**LightGBM**

In [138]:
from lightgbm import LGBMRegressor
import lightgbm as lgb

In [139]:
params = {
    'objective': 'regression',
    'metric': ['l1', 'l2'],
    'learning_rate': 0.005,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'verbose': 0,
    'max_depth': 8,
    'num_leaves': 128,
    'max_bin': 512,
    'num_boost_round': 100000,
    'early_stopping_round': 500,
}

In [140]:
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

In [141]:
model = lgb.train(params, train_data, valid_sets=[test_data])
y_pred = model.predict(X_test, num_iteration=model.best_iteration)
y_pred

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:300: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

array([12446.45939201, 36701.86679233, 13973.69423048, ...,
       21082.50465953, 12049.75286494, 11386.70972209])

In [142]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse

161090.41355327112

In [143]:
R2= r2_score(y_test, y_pred)
R2

0.15813113858886263

**CATBoost**

In [22]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.5 MB/s eta 0:00:00


In [23]:
from catboost import CatBoostRegressor
cbr = CatBoostRegressor(iterations=100,
                        depth=5,
                        learning_rate=0.01,
                        loss_function='RMSE',
                        verbose=0)
cbr.fit(X_train, y_train)
y_pred = cbr.predict(X_test)
y_pred

array([21938.59528852, 22822.03304696, 22289.9967797 , ...,
       22724.67074334, 21330.8328157 , 22009.54174211])

In [24]:
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
rmse

168030.29160407907

In [25]:
R2=r2_score(y_test, y_pred)
R2

0.08403218119015221

**XGBoost**

In [26]:
from sklearn.ensemble import GradientBoostingRegressor

In [27]:
from xgboost import XGBRegressor
xgr = XGBRegressor()
xgr.fit(X_train, y_train)
y_pred = xgr.predict(X_test)
y_pred

array([10549.777 , 22107.77  , 41163.617 , ..., -3785.9763,  7992.927 ,
       15691.191 ], dtype=float32)

In [28]:
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
rmse

96330.65284645274

In [29]:
R2=r2_score(y_test, y_pred)
R2

0.698953168511524

**Random Forest**

In [30]:
from sklearn.ensemble import RandomForestRegressor

In [31]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
y_pred

<ipython-input-31-dba6e765d798>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_model.fit(X_train, y_train)


array([ 8212.0152    , 21837.86253333, 11651.3582    , ...,
       10147.2485    ,  9698.1255    ,  8005.57284381])

In [32]:
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
rmse

139021.20082560432

In [33]:
R2=r2_score(y_test, y_pred)
R2

0.37300060108141697

 **KFold cross validation**

In [34]:
from sklearn.model_selection import KFold
from lightgbm import Dataset

In [35]:
num_folds = 8
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
validation_results = []

In [36]:
for train_idx, val_idx in kf.split(X):
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    train_data = Dataset(X_train, label=y_train)
    valid_data = Dataset(X_val, label=y_val)

**LightGBM**

In [37]:
params = {
    'objective': 'regression',
    'metric': ['l1', 'l2'],
    'learning_rate': 0.005,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'verbose': 0,
    'max_depth': 8,
    'num_leaves': 128,
    'max_bin': 512,
    'num_boost_round': 100000,
    'early_stopping_round': 500,
}

In [38]:
model = lgb.train(params, train_data, valid_sets=[valid_data])
y_pred = model.predict(X_val, num_iteration=model.best_iteration)
y_pred

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py:300: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

array([  8899.41194202, 151230.62520091, 130162.55193151,  16851.60145287,
        14322.84718984,   9234.21273129,  11631.46940224,   9980.36162784,
        10826.41189245,  10859.48252458,   4928.62940309,   8470.50496821,
       290771.79075576,  12699.13640536,   9311.85800385,  10343.02953847,
        10343.02953847,  67228.68818357,   8907.67452262,  12373.13362293,
         3626.68955176,   8107.00056285, 114660.31477741,  10077.55101283,
         9555.60209622,   9381.36740296,  12142.9761704 ,   8742.56571144,
         2714.08809499,  99944.15898734,  21274.89491138,  11835.51214684,
       405733.1704393 ,  10395.55543139, 128175.17384173, 112415.99622115,
        51522.44625942,  51957.97093238, 343824.45050345,  14729.29774036,
       -13417.5906364 ,   8864.55098677,   9354.39368081,  10759.14858357,
        10835.27587781,  75584.21716908, 204868.71413868,  26518.72415053,
         9481.28683574,   9090.69675738,   7564.71733779, 120218.45463759,
        19800.79105641,  

In [39]:
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
rmse

80390.51993996183

In [40]:
R2= r2_score(y_val, y_pred)
R2

0.2940336601580126

**CatBoost**

In [41]:
cbr.fit(X_train, y_train)
y_pred = cbr.predict(X_val)
y_pred

array([ 21283.14854882,  80820.27454458,  79358.60429604,  36316.50918251,
        20904.94607104,  20657.65572556,  23437.43363556,  20841.81442711,
        25729.74807602,  21423.22116765,  38788.54316797,  39542.32499005,
       154446.63591913,  22272.13291672,  20680.83127589,  22272.13291672,
        22272.13291672,  41764.67361227,  20723.01814748,  20697.71144733,
        35846.51649695,  20926.65867473,  79796.14420653,  20680.83127589,
        20638.12961929,  20671.19081384,  24595.50268323,  20667.08608014,
        38854.11569736,  80200.13832194,  24110.51538458,  20775.58663333,
       201904.37460197,  21178.43339723,  80206.11413076, 106189.58055879,
        78440.44305667,  23326.20835778, 153636.96230133,  26614.07418939,
        23072.10881814,  20908.12569425,  20715.77843809,  20997.37203976,
        20753.79943976,  77758.82486647, 129129.62573016,  22216.63636607,
        20789.98709347,  20631.8911127 ,  20875.59076108, 112288.32996055,
        20960.61802907,  

In [42]:
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
rmse

85585.80433933921

In [43]:
R2= r2_score(y_val, y_pred)
R2

0.19983823980000825

**XGBoost**

In [44]:
xgr = XGBRegressor()
xgr.fit(X_train, y_train)
y_pred = xgr.predict(X_val)

In [45]:
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
rmse

82989.26262181072

In [46]:
R2= r2_score(y_val, y_pred)
R2

0.24765310821546027

**Random Forest**

In [47]:
rf_model.fit(X_train, y_train)
y_pred= rf_model.predict(X_val)
y_pred

<ipython-input-47-02e923c3f196>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_model.fit(X_train, y_train)


array([1.45571187e+04, 3.08764929e+05, 4.82560433e+05, 1.95686472e+04,
       3.70026805e+04, 8.86025610e+03, 1.76298594e+04, 1.49043841e+04,
       2.78411732e+04, 1.05756215e+04, 2.92422140e+03, 2.93972150e+03,
       1.08649818e+05, 3.01140260e+03, 5.66338090e+03, 3.38998190e+03,
       3.38998190e+03, 1.87171440e+04, 1.04843598e+04, 9.91813250e+03,
       9.78643720e+03, 4.27719690e+03, 2.48688367e+05, 4.35898266e+04,
       4.62425930e+03, 9.90202060e+03, 2.49187360e+03, 5.67125470e+03,
       9.09707370e+03, 1.12446880e+05, 1.15816983e+04, 7.33207420e+03,
       2.94071145e+05, 3.00951680e+03, 5.73426131e+04, 2.09298100e+05,
       3.44289623e+04, 5.67170193e+04, 4.90192227e+05, 7.08193760e+03,
       6.38723830e+03, 1.12697440e+03, 3.49537200e+03, 7.80233120e+03,
       1.35947917e+04, 8.11212277e+04, 2.64333569e+05, 1.68113497e+04,
       6.88498460e+03, 5.81083580e+03, 3.20877440e+03, 2.06095919e+05,
       7.35072550e+03, 9.40272800e+03, 5.68668129e+04, 8.54556580e+03,
      

In [48]:
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
rmse

73648.4789334549

In [49]:
R2= r2_score(y_val, y_pred)
R2

0.4074815432697626

**Ensemble Kfold**

**XGBoost-CatBoost Ensembled KFold**

In [50]:
from sklearn.ensemble import VotingRegressor

In [51]:
ensemble = VotingRegressor(estimators=[('xgb', xgr), ('catboost', cbr)], weights=[0.5, 0.5])
ensemble.fit(X_train, y_train)
y_pred = ensemble.predict(X_val)
y_pred

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_voting.py:597: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([ 17534.76226269, 217229.40289729, 269359.97402302,   4775.93134907,
        20405.72987146,  12573.97337059,  19642.88966934,  12211.14647137,
        19705.44337395,  16575.61937289,  19310.44628186,  22998.103413  ,
       116900.14217831,  12296.22905602,  12801.31773755,  10427.48155846,
        10427.48155846,  25282.31190379,  12245.52433253,  16728.91285257,
        13163.93110004,  12934.52074362, 119243.97054076,  54452.44298169,
        13829.60436042,  15315.02900067,  14461.16491583,  11981.46125296,
        20292.56791948, 137547.55353597,  20411.56628604,  13766.03110963,
       146505.49589474,  10249.84499451,  71969.50237788, 153075.07934189,
        49294.54867677,  98254.54167889, 313788.09052567,  33420.30857907,
        15652.46993641,  15121.18980025,  13872.26446318,  18404.63963316,
        12334.9395148 ,  72450.84993324, 138466.18005258,  13938.51837835,
        13365.31581236,  15720.97778291,   9373.50900359, 138422.25873028,
         9161.98211024,  

In [52]:
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
rmse

76912.80055833982

In [53]:
R2= r2_score(y_val, y_pred)
R2

0.3537931312960423

**XGboost-lightgbm**

In [57]:
lgb = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.05, max_depth=5)
ensemble = VotingRegressor(estimators=[('xgb', xgr), ('lightgbm', lgb)], weights=[0.5, 0.5])
ensemble.fit(X_train, y_train)
y_pred = ensemble.predict(X_val)
y_pred

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_voting.py:597: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000804 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 783
[LightGBM] [Info] Number of data points in the train set: 6835, number of used features: 54
[LightGBM] [Info] Start training from score 37302.732862
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further 

array([ 5.60661287e+03,  2.61813814e+05,  3.02480765e+05,  1.67592714e+04,
        1.47133618e+04,  4.16906295e+03,  1.45689831e+04,  5.66164678e+03,
        1.18494384e+04,  8.36514919e+03,  4.98734832e+02,  1.64891943e+03,
        2.54123364e+05,  5.22849795e+03,  6.68554950e+03,  3.48669097e+03,
        3.48669097e+03,  2.70212742e+04,  6.23102502e+03,  1.13910734e+04,
       -4.49999787e+03,  5.77056704e+03,  1.48188341e+05,  4.83956032e+04,
        7.55176982e+03,  8.67248932e+03,  7.11084847e+03,  4.82788142e+03,
        1.44807020e+03,  1.47161514e+05,  2.38470472e+04,  6.57613043e+03,
        3.38264490e+05,  7.30434149e+03,  9.74640683e+04,  1.70678252e+05,
        3.38678856e+04,  1.16149728e+05,  4.73379956e+05,  2.65680289e+04,
       -1.08369253e+04,  8.93223178e+03,  7.35104518e+03,  1.25836835e+04,
        6.16666220e+03,  5.98201716e+04,  2.08801316e+05,  2.47770263e+04,
        6.95422541e+03,  9.15461577e+03,  3.02414586e+03,  1.35661238e+05,
        3.23495948e+03,  

In [58]:
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
rmse

75641.0820625146

In [59]:
R2= r2_score(y_val, y_pred)
R2

0.37498594265677954

**LightGBM-CatBoost**

In [60]:
#lgb = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.05, max_depth=5)
ensemble = VotingRegressor(estimators=[('catboost', cbr), ('lightgbm', lgb)], weights=[0.5, 0.5])

ensemble.fit(X_train, y_train)
y_pred = ensemble.predict(X_val)

y_pred

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_voting.py:597: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000593 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 783
[LightGBM] [Info] Number of data points in the train set: 6835, number of used features: 54
[LightGBM] [Info] Start training from score 37302.732862
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further 

array([  9354.99915893, 125404.68567114, 112479.39555831,  48299.84920548,
        15212.57800081,  12252.74530344,  18363.52709503,  14292.31473194,
        17873.74311606,  13212.75098192,  19976.83171767,  18193.14100846,
       291669.85766291,  15204.40180738,  14565.06303896,  15331.34232659,
        15331.34232659,  43503.6358646 ,  14708.51883126,  15359.87203395,
        18182.58752752,  13762.70496826, 108740.51422409,  14623.99149187,
        14360.29507986,  14028.65113712,  17245.18623332,  13513.50624398,
        20009.61798237,  89814.0985255 ,  27545.9962844 ,  13585.68595864,
       393663.36861807,  18232.92989454, 105700.68006817, 123792.75290751,
        63013.77997398,  41221.39448139, 313228.82801707,  19761.79452823,
        -3417.28643998,  14719.16767782,  14194.55915031,  15176.4159461 ,
        14585.52212995,  65128.14657522, 199464.76157403,  33055.14427279,
        14378.89669125,  14065.52909929,  14526.22761795, 109527.30875486,
        15033.59539946,  

In [61]:
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
rmse

80762.61415856835

In [62]:
R2= r2_score(y_val, y_pred)
R2

0.28748328765675135

**CatBoost-LightGBM-XgBoost Ensembled KFold**

In [63]:
ensemble = VotingRegressor(estimators=[('xgb', xgr), ('catboost', cbr), ('lightgbm', lgb)], weights=[0.3, 0.3, 0.4])
ensemble.fit(X_train, y_train)
y_pred = ensemble.predict(X_val)
y_pred

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_voting.py:597: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000614 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 783
[LightGBM] [Info] Number of data points in the train set: 6835, number of used features: 54
[LightGBM] [Info] Start training from score 37302.732862
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further 

array([  9491.59726523, 198333.28045745, 219856.05914204,  26978.83450082,
        16051.5218951 ,   9083.51797488,  17101.5820234 ,  10423.81389753,
        15830.36128681,  11946.28394221,  12052.31587607,  12536.44485854,
       241697.31706966,  10632.40571282,  11060.50856335,   9612.70962966,
         9612.70962966,  33266.42638904,  10824.92240554,  14046.16075977,
         8105.82208326,  10400.21295088, 126620.33602112,  36098.32647216,
        11530.74683243,  12143.46198456,  12634.64686287,   9732.8473149 ,
        12641.58885864, 122299.7556132 ,  24639.53064531,  10817.93277936,
       322072.24259051,  12264.87755345,  95659.79982896, 148403.41770763,
        48611.57596258,  82599.35724933, 377401.13180852,  25215.99109427,
        -2571.19071739,  12484.79774471,  11392.69462292,  14784.96772087,
        10767.86163694,  64469.49727353, 190999.66699871,  25920.57189882,
        11206.31200303,  12432.2535041 ,   8894.85119208, 125759.87025783,
         9139.81837408,  

In [64]:
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
rmse

75930.77568898759

In [65]:
R2= r2_score(y_val, y_pred)
R2

0.37018936109673295

**Random Forest-XGBoost**

In [66]:
ensemble = VotingRegressor(estimators=[('xgb', xgr), ('RandomForest', rf_model)], weights=[0.5, 0.5])
ensemble.fit(X_train, y_train)
y_pred = ensemble.predict(X_val)
y_pred

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_voting.py:597: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([ 1.41717473e+04,  3.31201730e+05,  4.70960888e+05, -3.59799964e+03,
        2.84545971e+04,  6.67527356e+03,  1.67391026e+04,  9.24243131e+03,
        2.07611559e+04,  1.11518195e+04,  1.37828540e+03,  4.69680167e+03,
        9.40017331e+04,  2.66586390e+03,  5.29259255e+03,  9.86406050e+02,
        9.86406050e+02,  1.37585471e+04,  7.12619516e+03,  1.13391234e+04,
        1.33891452e+02,  4.60978986e+03,  2.03690082e+05,  6.59069406e+04,
        5.82266920e+03,  9.93044389e+03,  3.40935037e+03,  4.48354556e+03,
        5.41404692e+03,  1.53670925e+05,  1.41471577e+04,  7.04427489e+03,
        1.92588881e+05,  1.16538670e+03,  6.05377519e+04,  2.04629339e+05,
        2.72888083e+04,  1.14949947e+05,  4.82065723e+05,  2.36542403e+04,
        7.31003468e+03,  5.23061415e+03,  5.26206124e+03,  1.18071192e+04,
        8.75543564e+03,  7.41320513e+04,  2.06068152e+05,  1.12358751e+04,
        6.41281457e+03,  8.31045013e+03,  5.40100823e+02,  1.85326053e+05,
        2.35703585e+03,  

In [67]:
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
rmse

75100.96813319168

In [68]:
R2= r2_score(y_val, y_pred)
R2

0.38387988100966397

**Random Forest-CatBoost**

In [69]:
ensemble = VotingRegressor(estimators=[('catboost', cbr), ('RandomForest', rf_model)], weights=[0.5, 0.5])
ensemble.fit(X_train, y_train)
y_pred = ensemble.predict(X_val)
y_pred

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_voting.py:597: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([ 17920.13362441, 194792.60187229, 280959.51849802,  27942.57819125,
        28953.81328552,  14758.95591278,  20533.64651778,  17873.09926355,
        26785.46063801,  15999.42133383,  20856.38228398,  21241.02324503,
       131548.22685956,  12641.76775836,  13172.10608794,  12831.05740836,
        12831.05740836,  30240.90880613,  15603.68897374,  15307.92197367,
        22816.47684847,  12601.92778737, 164242.25565326,  32135.32893794,
        12631.19445964,  15286.60570692,  13543.68814162,  13169.17039007,
        23975.59469868,  96323.50931097,  17846.10684229,  14053.83041666,
       247987.75995099,  12093.97509861,  68774.36361538, 157743.84047939,
        56434.70267834,  40021.61382889, 321914.59470067,  16848.00589469,
        14729.67355907,  11017.55004713,  12105.57521904,  14399.85161988,
        17174.29556988,  79440.02628324, 196731.59721508,  19513.99305179,
        13837.48584673,  13221.36345635,  12042.18258054, 159192.12463028,
        14155.67176454,  

In [70]:
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
rmse

74620.18643991023

In [71]:
R2= r2_score(y_val, y_pred)
R2

0.3917431912309842

**Random Forest-LightGBM**

In [72]:
ensemble = VotingRegressor(estimators=[('lightgbm', lgb), ('RandomForest', rf_model)], weights=[0.5, 0.5])
ensemble.fit(X_train, y_train)
y_pred = ensemble.predict(X_val)
y_pred

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_voting.py:597: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000845 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 783
[LightGBM] [Info] Number of data points in the train set: 6835, number of used features: 54
[LightGBM] [Info] Start training from score 37302.732862
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further 

array([ 5.99198423e+03,  2.39377013e+05,  3.14080310e+05,  3.99259182e+04,
        2.32614452e+04,  6.35404549e+03,  1.54597400e+04,  1.13235996e+04,
        1.89294557e+04,  7.78895115e+03,  2.04467083e+03, -1.08160737e+02,
        2.68771449e+05,  5.57403665e+03,  7.05633785e+03,  5.89026682e+03,
        5.89026682e+03,  3.19798711e+04,  9.58918966e+03,  9.97008256e+03,
        5.15254788e+03,  5.43797408e+03,  1.93186626e+05,  2.60784892e+04,
        6.35335992e+03,  8.64406603e+03,  6.19337169e+03,  6.01559055e+03,
        5.13109698e+03,  1.05937470e+05,  2.12815877e+04,  6.86392974e+03,
        4.39746754e+05,  9.14847160e+03,  9.42689296e+04,  1.75347013e+05,
        4.10080396e+04,  5.79168000e+04,  4.81506460e+05,  9.99572623e+03,
       -1.17597217e+04,  4.82859203e+03,  5.58435593e+03,  8.57889553e+03,
        1.10060183e+04,  6.68093480e+04,  2.67066733e+05,  3.03525010e+04,
        7.42639544e+03,  6.65500144e+03,  5.69281944e+03,  1.56431103e+05,
        8.22864913e+03,  

In [73]:
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
rmse

73961.30181686296

In [74]:
R2= r2_score(y_val, y_pred)
R2

0.4024373935424268

**Random forest-catboost-lightgbm**

In [75]:
ensemble = VotingRegressor(estimators=[('RandomForest', rf_model), ('catboost', cbr), ('lightgbm', lgb)], weights=[0.3, 0.3, 0.4])
ensemble.fit(X_train, y_train)
y_pred = ensemble.predict(X_val)
y_pred

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_voting.py:597: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000615 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 783
[LightGBM] [Info] Number of data points in the train set: 6835, number of used features: 54
[LightGBM] [Info] Start training from score 37302.732862
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further 

array([ 9.72282008e+03,  1.84871200e+05,  2.26815786e+05,  4.08788226e+04,
        2.11803719e+04,  1.03945075e+04,  1.76360361e+04,  1.38209856e+04,
        2.00783716e+04,  1.16005651e+04,  1.29798775e+04,  1.14821968e+04,
        2.50486168e+05,  1.08397289e+04,  1.12829816e+04,  1.10548551e+04,
        1.10548551e+04,  3.62415845e+04,  1.28398212e+04,  1.31935662e+04,
        1.38973495e+04,  1.02006572e+04,  1.53619307e+05,  2.27080580e+04,
        1.08117009e+04,  1.21264080e+04,  1.20841608e+04,  1.04454728e+04,
        1.48514049e+04,  9.75653291e+04,  2.31002550e+04,  1.09906124e+04,
        3.82961601e+05,  1.33713556e+04,  9.37427166e+04,  1.51204674e+05,
        5.28956684e+04,  4.76596005e+04,  3.82277034e+05,  1.52726095e+04,
       -3.12486854e+03,  1.00226139e+04,  1.03326811e+04,  1.23820949e+04,
        1.36714753e+04,  6.86630031e+04,  2.25958917e+05,  2.92658567e+04,
        1.14896140e+04,  1.09324849e+04,  1.04960553e+04,  1.38221790e+05,
        1.21360322e+04,  

In [76]:
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
rmse

75467.6554992585

In [77]:
R2= r2_score(y_val, y_pred)
R2

0.37784866689190644

**Random forest-XGboost-lightgbm KFold Ensembled**

In [78]:
ensemble = VotingRegressor(estimators=[('RandomForest', rf_model), ('xgb', xgr), ('lightgbm', lgb)], weights=[0.3, 0.3, 0.4])
ensemble.fit(X_train, y_train)
y_pred = ensemble.predict(X_val)
y_pred

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_voting.py:597: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000496 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 783
[LightGBM] [Info] Number of data points in the train set: 6835, number of used features: 54
[LightGBM] [Info] Start training from score 37302.732862
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further 

array([ 7.47378831e+03,  2.66716677e+05,  3.40816608e+05,  2.19544759e+04,
        2.08808422e+04,  5.54429809e+03,  1.53593098e+04,  8.64258480e+03,
        1.64637888e+04,  8.69200404e+03,  1.29301935e+03,  1.55566381e+03,
        2.27958272e+05,  4.85418662e+03,  6.55527345e+03,  3.94806432e+03,
        3.94806432e+03,  2.63521675e+04,  7.75332490e+03,  1.08122871e+04,
        2.87798294e+02,  5.40537442e+03,  1.77288003e+05,  4.29710251e+04,
        6.72658574e+03,  8.91271092e+03,  6.00355814e+03,  5.23409790e+03,
        3.71447626e+03,  1.31973778e+05,  2.08808855e+04,  6.78487905e+03,
        3.49722274e+05,  6.81420257e+03,  8.88007495e+04,  1.79335974e+05,
        3.54081317e+04,  9.26166005e+04,  4.78367711e+05,  1.93563501e+04,
       -7.57665187e+03,  6.55045236e+03,  6.22657269e+03,  1.08264555e+04,
        8.62015932e+03,  6.54782181e+04,  2.31560850e+05,  2.42989859e+04,
        7.03481125e+03,  7.98593691e+03,  3.59480628e+03,  1.53902147e+05,
        5.05685062e+03,  

In [79]:
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
rmse

73411.94517402661

In [80]:
R2= r2_score(y_val, y_pred)
R2

0.41128136540239046

**Random forest-catboost-XGBoost KFold Ensembled**

In [81]:
ensemble = VotingRegressor(estimators=[('RandomForest', rf_model), ('xgb', xgr), ('catboost', cbr)], weights=[0.3, 0.3, 0.4])
ensemble.fit(X_train, y_train)
y_pred = ensemble.predict(X_val)
y_pred

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_voting.py:597: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([ 17016.3078225 , 231049.14795283, 314319.97465342,  12367.80388769,
        25434.73667998,  12268.22642491,  19418.43498516,  13882.18455553,
        22748.59279197,  15260.3801905 ,  16342.38850591,  18635.0109968 ,
       118179.6942389 ,  10508.37150528,  11447.88804012,   9500.69679675,
         9500.69679675,  24960.9977035 ,  12564.92435427,  15082.55860628,
        14418.94146972,  11136.53738364, 154132.50687511,  47816.49689661,
        11748.85336818,  14226.74266179,  11883.81129782,  10956.96176979,
        18790.07443143, 124282.61004377,  18132.50080008,  12536.79958911,
       196315.07858704,   9170.60537643,  68405.0967698 , 165253.43578101,
        47749.46220173,  78300.45163311, 350694.21867553,  24838.17384638,
        13614.86433366,  11501.61876958,  11443.54812172,  15483.22034387,
        13554.78116286,  75582.76075659, 175292.74121456,  15628.17958487,
        12163.68357676,  13239.02652102,   8674.29679826, 156110.96402422,
         9798.46871905,  

In [82]:
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
rmse

74022.32198777272

In [83]:
R2= r2_score(y_val, y_pred)
R2

0.4014509746142243

**Auto ML**

**Auto Keras**

In [84]:
!pip install autokeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.1/590.1 kB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 70.9 MB/s eta 0:00:00


In [85]:
import autokeras as ak
from tensorflow.keras import layers

Using TensorFlow backend


In [86]:
reg = ak.StructuredDataRegressor(max_trials=5, overwrite=True)

In [87]:
#reg = ak.StructuredDataRegressor(max_trials=8)
reg.fit(X_train, y_train, epochs=8)
#y_pred = reg.evaluate(X_test, y_test)
y_pred = reg.predict(X_test)
# rf_model.fit(X_train, y_train)
# y_pred = rf_model.predict(X_test)
y_pred

Trial 5 Complete [00h 01m 57s]
val_loss: 5333737348792320.0

Best val_loss So Far: 454456541184.0
Total elapsed time: 00h 11m 33s
Epoch 1/8
214/214 [==============================] - 6s 15ms/step - loss: 36463853568.0000 - mean_squared_error: 36463853568.0000
Epoch 2/8
214/214 [==============================] - 2s 9ms/step - loss: 36451201024.0000 - mean_squared_error: 36451201024.0000
Epoch 3/8
214/214 [==============================] - 2s 9ms/step - loss: 36401065984.0000 - mean_squared_error: 36401065984.0000
Epoch 4/8
214/214 [==============================] - 2s 9ms/step - loss: 36294033408.0000 - mean_squared_error: 36294033408.0000
Epoch 5/8
214/214 [==============================] - 2s 9ms/step - loss: 36120276992.0000 - mean_squared_error: 36120276992.0000
Epoch 6/8
214/214 [==============================] - 4s 16ms/step - loss: 35883761664.0000 - mean_squared_error: 35883761664.0000
Epoch 7/8
214/214 [==============================] - 2s 9ms/step - loss: 35600093184.0000 - me

array([[ 8822.417   ],
       [20422.7     ],
       [11893.761   ],
       ...,
       [   50.617256],
       [13576.918   ],
       [20125.584   ]], dtype=float32)

In [89]:
y_pred

array([[ 8822.417   ],
       [20422.7     ],
       [11893.761   ],
       ...,
       [   50.617256],
       [13576.918   ],
       [20125.584   ]], dtype=float32)

In [90]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse

175544.90435023446

**H2O**

In [95]:
import h2o
from h2o.automl import H2OAutoML

In [93]:
pip install H2O

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.4/257.4 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for H2O: filename=h2o-3.44.0.1-py2.py3-none-any.whl size=257484150 sha256=c0675d4438beaa73fd2ef754ce5fa587422e6a45589a7cc234c1d0f88c08010e
  Stored in directory: /root/.cache/pip/wheels/d9/9b/ca/7345b72d17e1e17da37239d70631c3214ec9e541b0c9e700e2
Successfully built H2O


In [96]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.20.1" 2023-08-24; OpenJDK Runtime Environment (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpmy64emt4
  JVM stdout: /tmp/tmpmy64emt4/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpmy64emt4/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.1
H2O_cluster_version_age:,10 days
H2O_cluster_name:,H2O_from_python_unknownUser_0o8dqt
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [97]:
X_train_h2o = h2o.H2OFrame(X_train)
y_train_h2o = h2o.H2OFrame(y_train)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [147]:
aml = H2OAutoML(max_models=25, seed=1)
aml.train(x=X_train_h2o.columns, y=y_train_h2o.columns[0], training_frame=X_train_h2o)

AutoML progress: |
08:49:33.295: _train param, Dropping bad and constant columns: [C156, C49, C180, C93, C78, C94]

██
08:50:05.551: _train param, Dropping bad and constant columns: [C156, C49, C180, C93, C78, C94]


08:50:09.98: _train param, Dropping bad and constant columns: [C156, C49, C180, C93, C78, C94]

███
08:50:45.390: _train param, Dropping bad and constant columns: [C156, C49, C180, C93, C78, C94]


08:50:56.730: _train param, Dropping bad and constant columns: [C156, C49, C180, C93, C78, C94]

███
08:51:40.958: _train param, Dropping bad and constant columns: [C156, C49, C180, C93, C78, C94]

█
08:52:12.636: _train param, Dropping bad and constant columns: [C156, C49, C180, C93, C78, C94]

█
08:52:44.328: _train param, Dropping bad and constant columns: [C156, C49, C180, C93, C78, C94]

██
08:53:25.862: _train param, Dropping bad and constant columns: [C156, C49, C180, C93, C78, C94]


08:53:37.972: _train param, Dropping bad and constant columns: [C156, C49, C180, C93, C7

key,value
Stacking strategy,cross_validation
Number of base models (used / total),8/25
# GBM base models (used / total),1/9
# XGBoost base models (used / total),4/8
# DRF base models (used / total),2/2
# GLM base models (used / total),0/1
# DeepLearning base models (used / total),1/5
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


In [148]:
y_test_h2o=h2o.H2OFrame(y_test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [149]:
leaderboard = aml.leaderboard
print(leaderboard)
best_model = aml.leader

model_id                                                   rmse     mse      mae       rmsle    mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_2_20231027_84933     567.14   321647  438.899  nan                           321647
StackedEnsemble_BestOfFamily_1_AutoML_2_20231027_84933  567.336  321870  439.222  nan                           321870
DRF_1_AutoML_2_20231027_84933                           578.386  334530  451.495    0.542227                    334530
XGBoost_grid_1_AutoML_2_20231027_84933_model_2          592.796  351408  454.262  nan                           351408
GBM_grid_1_AutoML_2_20231027_84933_model_1              597.935  357526  481.107    0.561963                    357526
XGBoost_grid_1_AutoML_2_20231027_84933_model_4          601.903  362288  470.563    0.55631                     362288
GBM_4_AutoML_2_20231027_84933                           603.728  364488  487.669    0.568293                    364488
GBM_3_AutoML_2_20231027_84933                   

In [150]:
perf = best_model.model_performance(y_test_h2o)
perf

ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 32037451043.569748
RMSE: 178990.08643936052
MAE: 35215.88943712186
RMSLE: 2.332328003894506
Mean Residual Deviance: 32037451043.569748
R^2: -0.039352835832995314
Null degrees of freedom: 1562
Residual degrees of freedom: 1554
Null deviance: 50063463700444.06
Residual deviance: 50074535981099.516
AIC: 42264.83975383828

In [102]:
h2o_pred=best_model.predict(y_test_h2o).as_data_frame()

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


/usr/local/lib/python3.10/dist-packages/h2o/job.py:81: UserWarning: Test/Validation dataset is missing column 'C2': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.10/dist-packages/h2o/job.py:81: UserWarning: Test/Validation dataset is missing column 'C3': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.10/dist-packages/h2o/job.py:81: UserWarning: Test/Validation dataset is missing column 'C4': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.10/dist-packages/h2o/job.py:81: UserWarning: Test/Validation dataset is missing column 'C5': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.10/dist-packages/h2o/job.py:81: UserWarning: Test/Validation dataset is missing column 'C6': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.10/dist-packages/h2o/job.py:81: UserWarning: Test/Validation dataset is missing column 'C7': substituting in a column of NaN
  warnings.warn(w

In [103]:
h2o_pred=np.array(h2o_pred)
h2o_pred

array([[1498.08330925],
       [1498.08330925],
       [1498.08330925],
       ...,
       [1498.08330925],
       [1498.08330925],
       [1498.08330925]])

In [104]:
y_pred = best_model.predict(y_test_h2o)
rmse = perf.rmse()
rmse

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


/usr/local/lib/python3.10/dist-packages/h2o/job.py:81: UserWarning: Test/Validation dataset is missing column 'C2': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.10/dist-packages/h2o/job.py:81: UserWarning: Test/Validation dataset is missing column 'C3': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.10/dist-packages/h2o/job.py:81: UserWarning: Test/Validation dataset is missing column 'C4': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.10/dist-packages/h2o/job.py:81: UserWarning: Test/Validation dataset is missing column 'C5': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.10/dist-packages/h2o/job.py:81: UserWarning: Test/Validation dataset is missing column 'C6': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.10/dist-packages/h2o/job.py:81: UserWarning: Test/Validation dataset is missing column 'C7': substituting in a column of NaN
  warnings.warn(w

178985.44732710312

In [ ]:
h2o.shutdown()

**TPOT**

In [105]:
pip install tpot

In [106]:
from tpot import TPOTRegressor

In [151]:
tpot = TPOTRegressor(generations=10, population_size=20, verbosity=2, random_state=42)
tpot.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Optimization Progress:   0%|          | 0/220 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -24910332270.263042

Generation 2 - Current best internal CV score: -24910332270.263042

Generation 3 - Current best internal CV score: -24910332270.263042

Generation 4 - Current best internal CV score: -24559257608.863716

Generation 5 - Current best internal CV score: -24559257608.863716

Generation 6 - Current best internal CV score: -24559257608.863716

Generation 7 - Current best internal CV score: -22498782137.093666

Generation 8 - Current best internal CV score: -22498782137.093666

Generation 9 - Current best internal CV score: -20158187371.438473

Generation 10 - Current best internal CV score: -17194617016.76498

Best pipeline: RandomForestRegressor(input_matrix, bootstrap=False, max_features=0.35000000000000003, min_samples_leaf=1, min_samples_split=3, n_estimators=100)


TPOTRegressor(generations=10, population_size=20, random_state=42, verbosity=2)

In [108]:
best_pipeline = tpot.fitted_pipeline_
y_pred = best_pipeline.predict(X_test)
y_pred

array([14619.95036025, 33176.92450662,  8320.0309646 , ...,
       27029.75465519,  8320.0309646 , 27029.75465519])

In [152]:
best_pipeline

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('lassolarscv', LassoLarsCV(normalize=False))])

In [109]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse

168069.94043845145

In [110]:
R2= r2_score(y_test, y_pred)
R2

0.08359986222287619

In [111]:
best_pipeline = tpot.fitted_pipeline_
y_pred = best_pipeline.predict(X_val)
y_pred

array([ 14619.95036025, 131065.41663524, 131065.41663524,  45203.05007314,
        14467.20081603,  14467.20081603,   8320.0309646 ,  14467.20081603,
        24865.23344595,  14467.20081603,  45203.05007314,  45203.05007314,
       147610.6191166 ,  16586.56741301,   6270.97434746,  16586.56741301,
        16586.56741301,  45203.05007314,  14467.20081603,  14467.20081603,
        45203.05007314,   8320.0309646 , 131065.41663524,   8320.0309646 ,
         8320.0309646 ,   8320.0309646 ,  24712.48390173,  14467.20081603,
        45203.05007314, 137212.58648667,  14467.20081603,  14467.20081603,
       167948.43574378,  16433.81786878, 131065.41663524, 147457.86957237,
       129016.3600181 ,  13295.54412202, 147610.6191166 ,  26831.85049871,
        10286.64801736,  14467.20081603,   6270.97434746,  14467.20081603,
         8320.0309646 , 137212.58648667, 149424.48662512,  33176.92450662,
         8320.0309646 ,   8320.0309646 ,   8320.0309646 , 147457.86957237,
         8320.0309646 ,  

In [112]:
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
rmse

87875.10484293311

In [113]:
R2= r2_score(y_val, y_pred)
R2

0.15645931915659206